In [4]:
import pandas as pd
import geopandas as gp


In [5]:
first = pd.read_csv("C:\\Users\\visitor\\PycharmProjects\\flora-biel-2022\\data\\obs_export_2022-05-08_23h20.csv",encoding = "utf-16",sep = "\t")
second = pd.read_csv("C:\\Users\\visitor\\PycharmProjects\\flora-biel-2022\\data\\obs_export_2022-05-08_23h21.csv",encoding = "utf-16",sep = "\t")
all_data = pd.concat([first,second],ignore_index = True)

In [6]:
info_cols = {'obs_id':'unique identifier','date':'date of survey','x':'coordinate gis', 'y':'coordinate gis', 'taxon.taxon_id':'unique species identifier for infoflora reference',
            'taxon.no_isfs': 'unique species identifier for infoflora reference','taxon_orig':'original species name selected','municipality.name' : 'name of municipality sample taken',
            'phenology' : 'stage of growth','locality_descript' : 'name of survey site','specimen_type' : 'if there is a photo','v_co_canton' : 'name of canton sample taken',
            'xy_precision' : 'accuracy of x,y coordinates','altitude_min' : 'altitude for observations','cover':'coverage of area with the species'}

columns_keep = ['obs_id', 'date','determinavit_cf', 'taxon.taxon_id', 'taxon_orig','municipality.id','municipality.name', 'x', 'y', 'phenology','cover','altitude_min','locality_descript','specimen_type', 'v_co_canton','xy_precision']
columns_try = info_cols.keys()
# make a new df out of those columns
ad = all_data[columns_try].copy()
ad.to_csv("C:\\Users\\visitor\\PycharmProjects\\flora-biel-2022\\output\\alldata.csv")